In [2]:
import time
import datetime
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import tushare as ts
ts.set_token('29eaf3bcac23df4c6d025de157ab2d53beead3391fbe6e83b4ebcb6c')
pro = ts.pro_api()

import mpl_finance as mpf
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.pylab import date2num
import matplotlib.gridspec as gridspec
import matplotlib as mpl


In [3]:
from mylab.stock.myread import *
from mylab.stock.myfeature import *
from mylab.stock.myplot import *
from mylab.stock.my3point import *


In [4]:
SAVE_CSV = True
LOCAL = not True

GET_DAILY = True
SELECT_STOCK = True

start_date = "20100101"
end_date = "20200430"

# get all Stocks Info

In [5]:
#获取基础信息数据，包括股票代码、名称、上市日期、退市日期等
pool_df = getStockBasic(LOCAL = False,noSTKC = True, list_data = "20180101")
# pool_df = pd.read_csv("./data/stock/blue_stock_rank_3.csv" )

print('获得上市股票总数：', len(pool_df)-1)
pool_df.head()

获得上市股票总数： 2564


,ts_code,symbol,name,area,industry,market,exchange,list_status,list_date,delist_date
0,000001.SZ,000001,平安银行,深圳,银行,主板,SZSE,L,19910403,None
1,000002.SZ,000002,万科A,深圳,全国地产,主板,SZSE,L,19910129,None
2,000004.SZ,000004,国农科技,深圳,生物制药,主板,SZSE,L,19910114,None
3,000005.SZ,000005,世纪星源,深圳,环境保护,主板,SZSE,L,19901210,None
4,000006.SZ,000006,深振业A,深圳,区域地产,主板,SZSE,L,19920427,None


# constuct dataset
## collect all ZT

In [6]:
from mylab.stock.myread import *

In [ ]:
# policy
start_date = "20100101"
end_date = "20200430"
# stock_code = "002499.SZ"
save_dir = "./data/deeplearning_data/ZT/"
data_len = 64

index_df = getIndexDaily("000001.SH" , start_date , end_date, LOCAL = True, market = "SSE" )
for i in range(len(pool_df)):  # len(pool_df)
    stock_code = pool_df.loc[i,"ts_code"]
    stock_name = pool_df.loc[i,"name"]
    try:
        # select df 
        stock_df = getStockDaily(stock_code,start_date, end_date, LOCAL = True)
        stock_df = mergeStockIndex(stock_df,index_df)
        stock_df["high_pct"] = (stock_df["high"]/stock_df["pre_close"]-1)*100
        select_df = stock_df.loc[stock_df.high_pct.values > 9.5, :]
#         select_df["trade_date"] = select_df["trade_date"].astype("str")
        if len(select_df) <= 2:
            continue
#         select_df = myMerge(select_df,stock_df, on = "trade_date", how = "left")
#         print(stock_code,stock_name,len(select_df))
    except Exception as err:
        print("read err:",stock_code,stock_name,err)
        
    try:
        for i in select_df.index.values:
            select_date = select_df.loc[i,"trade_date"]
            stock_ix = stock_df.loc[stock_df["trade_date"].values == select_date,:].index[0]
            df = stock_df.iloc[stock_ix-data_len+1:stock_ix+1,:]
            if select_df["close"][i] < select_df["high"][i]:
                df.to_csv(save_dir+"DB/"+stock_code+"-"+select_date+".csv",index = False)
            else:
                df.to_csv(save_dir+"ZT/"+stock_code+"-"+select_date+".csv",index = False)
        # drop nan and save
        print(stock_code,stock_name,len(select_df))
    except Exception as err:
        print(stock_code,stock_name,err)
print(df.shape)
df.head()

000001.SZ 平安银行 14
000002.SZ 万科A 33
000004.SZ 国农科技 72
000005.SZ 世纪星源 84
000006.SZ 深振业A 50
000007.SZ 全新好 54
000008.SZ 神州高铁 54
000009.SZ 中国宝安 68
000011.SZ 深物业A 77
000012.SZ 南玻A 43
000014.SZ 沙河股份 98
000016.SZ 深康佳A 77
000019.SZ 深粮控股 84
000020.SZ 深华发A 108
000021.SZ 深科技 57
000023.SZ 深天地A 72
000025.SZ 特力A 134
000026.SZ 飞亚达 40
000027.SZ 深圳能源 23
000028.SZ 国药一致 23
000029.SZ 深深房A 64
000030.SZ 富奥股份 29
000031.SZ 大悦城 61
000032.SZ 深桑达A 85
000034.SZ 神州数码 85
000035.SZ 中国天楹 29
000036.SZ 华联控股 56
000037.SZ 深南电A 86
000038.SZ 深大通 88
000039.SZ 中集集团 28
000040.SZ 东旭蓝天 56
000042.SZ 中洲控股 28
000045.SZ 深纺织A 65
000046.SZ 泛海控股 44
000048.SZ 京基智农 60
000049.SZ 德赛电池 70
000050.SZ 深天马A 79
000055.SZ 方大集团 59
000056.SZ 皇庭国际 65
000058.SZ 深赛格 91
000059.SZ 华锦股份 28
000060.SZ 中金岭南 49
000061.SZ 农产品 45
000062.SZ 深圳华强 92
000063.SZ 中兴通讯 36
000065.SZ 北方国际 69
000066.SZ 中国长城 81
000068.SZ 华控赛格 61
000069.SZ 华侨城A 20
000070.SZ 特发信息 86
000078.SZ 海王生物 70
000088.SZ 盐田港 45
000089.SZ 深圳机场 21
000090.SZ 天健集团 47
000096.SZ 广聚能源 61
000099.SZ 中信海直 50
0

002040.SZ 南京港 67
002041.SZ 登海种业 70
002042.SZ 华孚时尚 30
002043.SZ 兔宝宝 56
002044.SZ 美年健康 66
002045.SZ 国光电器 70
002046.SZ 轴研科技 60
002047.SZ 宝鹰股份 50
002048.SZ 宁波华翔 36
002049.SZ 紫光国微 79
002050.SZ 三花智控 43
002051.SZ 中工国际 38
002052.SZ 同洲电子 77
002053.SZ 云南能投 64
002054.SZ 德美化工 64
002055.SZ 得润电子 88
002056.SZ 横店东磁 61
002057.SZ 中钢天源 82
002058.SZ 威尔泰 72
002059.SZ 云南旅游 70
002060.SZ 粤水电 40
002061.SZ 浙江交科 46
002062.SZ 宏润建设 58
002063.SZ 远光软件 51
002064.SZ 华峰氨纶 42
002065.SZ 东华软件 45
002066.SZ 瑞泰科技 95
002067.SZ 景兴纸业 63
002068.SZ 黑猫股份 41
002069.SZ 獐子岛 32
002071.SZ 长城影视 93
002073.SZ 软控股份 35
002074.SZ 国轩高科 81
002075.SZ 沙钢股份 76
002078.SZ 太阳纸业 27
002079.SZ 苏州固锝 68
002080.SZ 中材科技 43
002081.SZ 金螳螂 30
002082.SZ 万邦德 42
002083.SZ 孚日股份 19
002084.SZ 海鸥住工 69
002085.SZ 万丰奥威 62
002087.SZ 新野纺织 32
002088.SZ 鲁阳节能 24
002090.SZ 金智科技 61
002091.SZ 江苏国泰 38
002092.SZ 中泰化学 24
002093.SZ 国脉科技 72
002094.SZ 青岛金王 71
002095.SZ 生意宝 119
002096.SZ 南岭民爆 53
002097.SZ 山河智能 37
002098.SZ 浔兴股份 112
002099.SZ 海翔药业 56
002100.SZ 天康生物 56
002101.SZ 广东鸿图 4

002547.SZ 春兴精工 92
002548.SZ 金新农 77
002549.SZ 凯美特气 68
002550.SZ 千红制药 34
002551.SZ 尚荣医疗 74
002552.SZ 宝鼎科技 99
002553.SZ 南方轴承 47
002554.SZ 惠博普 58
002555.SZ 三七互娱 78
002556.SZ 辉隆股份 79
002557.SZ 洽洽食品 12
002558.SZ 巨人网络 59
002559.SZ 亚威股份 57
002560.SZ 通达股份 73
002561.SZ 徐家汇 33
002562.SZ 兄弟科技 73
002563.SZ 森马服饰 21
002564.SZ 天沃科技 48
002565.SZ 顺灏股份 70
002566.SZ 益盛药业 26
002567.SZ 唐人神 51
002568.SZ 百润股份 77
002570.SZ 贝因美 38
002571.SZ 德力股份 49
002572.SZ 索菲亚 34
002573.SZ 清新环境 38
002574.SZ 明牌珠宝 22
002576.SZ 通达动力 54
002577.SZ 雷柏科技 67
002578.SZ 闽发铝业 46
002579.SZ 中京电子 81
002580.SZ 圣阳股份 61
002581.SZ 未名医药 95
002582.SZ 好想你 50
002583.SZ 海能达 51
002584.SZ 西陇科学 95
002585.SZ 双星新材 28
002587.SZ 奥拓电子 61
002588.SZ 史丹利 27
002589.SZ 瑞康医药 35
002590.SZ 万安科技 78
002591.SZ 恒大高新 54
002592.SZ 八菱科技 87
002593.SZ 日上集团 57
002594.SZ 比亚迪 57
002595.SZ 豪迈科技 23
002596.SZ 海南瑞泽 96
002597.SZ 金禾实业 33
002598.SZ 山东章鼓 52
002599.SZ 盛通股份 65
002600.SZ 领益智造 94
002601.SZ 龙蟒佰利 51
002602.SZ 世纪华通 54
002603.SZ 以岭药业 31
002605.SZ 姚记科技 65
002606.SZ 大连电瓷 78
00

600155.SH 华创阳安 70
600156.SH 华升股份 96
600157.SH 永泰能源 74
600158.SH 中体产业 93
600159.SH 大龙地产 47
600160.SH 巨化股份 63
600161.SH 天坛生物 40
600162.SH 香江控股 69
600163.SH 中闽能源 44
600165.SH 新日恒力 87
600166.SH 福田汽车 24
600167.SH 联美控股 34
600168.SH 武汉控股 45
600169.SH 太原重工 64
600170.SH 上海建工 34
600171.SH 上海贝岭 59
600172.SH 黄河旋风 65
600173.SH 卧龙地产 59
600175.SH 美都能源 65
600176.SH 中国巨石 40
600177.SH 雅戈尔 17
600178.SH 东安动力 57
600180.SH 瑞茂通 54
600183.SH 生益科技 34
600184.SH 光电股份 62
600185.SH 格力地产 65
600186.SH 莲花健康 57
600187.SH 国中水务 59
600188.SH 兖州煤业 48
600189.SH 吉林森工 34
600190.SH 锦州港 43
600191.SH 华资实业 73
600192.SH 长城电工 49
600195.SH 中牧股份 32
600196.SH 复星医药 21
600197.SH 伊力特 35
600198.SH 大唐电信 89
600199.SH 金种子酒 44
600200.SH 江苏吴中 81
600201.SH 生物股份 21
600202.SH 哈空调 36
600203.SH 福日电子 72
600206.SH 有研新材 84
600207.SH 安彩高科 89
600208.SH 新湖中宝 53
600210.SH 紫江企业 55
600211.SH 西藏药业 55
600213.SH 亚星客车 81
600215.SH 长春经开 58
600216.SH 浙江医药 29
600217.SH 中再资环 48
600218.SH 全柴动力 78
600219.SH 南山铝业 20
600220.SH 江苏阳光 56
600221.SH 海航控股 40
600222.SH 太龙药业 

In [ ]:
def minMaxScale(s, min_value = None, max_value = None):
    if not min_value :
        min_value = min(s)
    if not max_value :
        max_value = max(s)
    s = (s-min_value)/(max_value-min_value)
    return s,min_value,max_value

def scaleDf(df):
    # sacle price
    # stock
    df["high"],min_value,max_value =  minMaxScale(df["high"], min_value = 0, max_value = None)
    df["open"],min_value,max_value =  minMaxScale(df["open"], min_value, max_value)
    df["low"],min_value,max_value =  minMaxScale(df["low"], min_value, max_value)
    df["close"],min_value,max_value =  minMaxScale(df["close"], min_value, max_value)
    df["pre_close"],min_value,max_value =  minMaxScale(df["pre_close"], min_value, max_value)
    # index
    df["high_index"],min_value,max_value =  minMaxScale(df["high_index"], min_value = 0, max_value = None)
    df["open_index"],min_value,max_value =  minMaxScale(df["open_index"], min_value, max_value)
    df["low_index"],min_value,max_value =  minMaxScale(df["low_index"], min_value, max_value)
    df["close_index"],min_value,max_value =  minMaxScale(df["close_index"], min_value, max_value)
    df["pre_close_index"],min_value,max_value =  minMaxScale(df["pre_close_index"], min_value, max_value)
    # sacale pct_chg
    df["pct_chg"],min_value,max_value =  minMaxScale(df["pct_chg"], min_value = -11, max_value = 11)
    df["pct_chg_index"],min_value,max_value =  minMaxScale(df["pct_chg_index"], min_value = -11, max_value = 11)
    # scale vol
    df["vol"],min_value,max_value =  minMaxScale(df["vol"], min_value = 0, max_value = None)
    df["vol_index"],min_value,max_value =  minMaxScale(df["vol_index"], min_value = 0, max_value = None)
    # scale amount
    df["amount"],min_value,max_value =  minMaxScale(df["amount"], min_value = 0, max_value = None)
    df["amount_index"],min_value,max_value =  minMaxScale(df["amount_index"], min_value = 0, max_value = None)
    # scale turnover rate
    df["turnover_rate"],min_value,max_value =  minMaxScale(df["turnover_rate"], min_value = 0, max_value = 30)
    df.loc[df["turnover_rate"] > 1,"turnover_rate"] = 1
    # scale vol ratio
    df["volume_ratio"],min_value,max_value =  minMaxScale(df["volume_ratio"], min_value = 0, max_value = 8)
    df.loc[df["volume_ratio"].values > 1,"volume_ratio"] = 1
    return df
    

In [ ]:
data_len = 64
# cut off useful cols
# delete outliers
# scale
use_cols =[ 'open', 'high', 'low', 'close', 'pre_close', 'pct_chg', 'vol', 'amount', 'turnover_rate', 'volume_ratio',
            'open_index', 'high_index','low_index','close_index','pre_close_index','pct_chg_index','vol_index','amount_index']
# use info dataframe to store filename and label
info_df = pd.DataFrame()
info_df["file_name"] = np.nan
info_df["label"] = np.nan

cats = ["ZT/","DB/"]
for cat in cats:
    folder = "./data/deeplearning_data/ZT/"+cat
    for foldername,subfolders,filenames in os.walk(folder):
        for file in filenames:
            filename = "./data/deeplearning_data/train_data/"+cat+file
            df = pd.read_csv(folder+file)
            if len(df) == data_len:
                # cut cols
                try:
                    df = df.loc[:,use_cols]
                except:
                    continue                        
                df = scaleDf(df)
                df.to_csv(filename, index = False)
                # record
                if cat == "ZT/":
                    label = 1
                else:
                    label = 0
                info_df = info_df.append({"file_name":filename,"label":label},ignore_index=True)
                
info_df["label"]= info_df["label"].astype(int)
info_df.to_csv("./data/deeplearning_data/train_data/"+"info.csv",index = False)

## create dataset 

In [76]:
from torch.utils.data import Dataset

class MyDatasetZT(Dataset):
    def __init__(self, df, transform=None, target_transform=None):
        self.info_df = df
        self.imgs = df["file_name"].values
        self.labels = df["label"].values
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        filename = self.imgs[index]
        label = self.labels[index]
        img = pd.read_csv(fn)
        if self.transform is not None:
            img = self.transform(img)
        return img,label

    def __len__(self):
        return len(self.imgs)

In [ ]:
# test created dataset
from torch.utils.data import DataLoader
info_df = pd.read_csv("./data/deeplearning_data/train_data/"+"info.csv")
dataset_ZT = MyDatasetZT(info_df, transform=transforms.ToTensor())
data_loader = DataLoader(dataset_ZT, batch_size=100,shuffle=True)


In [ ]:
from torchvision import transforms, utils
def show_batch(imgs):
    grid = utils.make_grid(imgs)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))
    plt.title('Batch from dataloader')

for i, (batch_x, batch_y) in enumerate(data_loader):
    if(i<4):
        print(i, batch_x.size(),batch_y.size())
        show_batch(batch_x)
        plt.axis('off')
        plt.show()